In [1]:
# import library
!pip install nltk
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import re

# download APIs for natural language processing
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk

True

In [2]:
import warnings as wrn
wrn.filterwarnings('ignore')
# read data from CSV file
df = pd.read_csv('../data/raw/dataTotal.csv')
# df = pd.read_csv('marvelRanarok.csv', encoding="cp1252")

# print(df["sentiment"].value_counts())
review = df["review"].tolist()

# convert label from text to digit process
positives = df[df["sentiment"] == "positive"] # group all the positive comments
negatives = df[df["sentiment"] == "negative"] # group all the negative comments
neutrals = df[df["sentiment"] == "neutral"]   # group all the neutral comments
negatives["sentiment"] = -1 # convert text label to digit lable -1 for negative
positives["sentiment"] = 1  # convert text label to digit lable 1 for positive
neutrals["sentiment"] = 0   # convert text label to digit lable 0 for neutral


# df["sentiment"] = df["sentiment"].map({"positive" : 1, "negative" : -1, "neutral" : 0})
# sentiment = df["sentiment"].tolist()
# Combine pos, neg, neutral data into one DataFrame data using pd.concat()
data = pd.concat([positives,
                  neutrals,
                  negatives,
                 ],axis=0)

data.reset_index(inplace=True) # reset the index of each row

print(data["sentiment"].value_counts())
data.head()
# data.info()
# print(review)
# print(sentiment)
# print(df.to_string())
# print(df.loc[2, "review"])
# print(df.info())
# df.head()
# pd.options.display.max_rows = 9999
# print(pd.options.display.max_rows)
# csv_size = len(df)


sentiment
-1    139
 1    125
 0     78
Name: count, dtype: int64


,index,review,sentiment
0,0,8 years later and still one of MARVEL’s best t...,1
1,1,Crazy how the line “he is a friend from work” ...,1
2,2,Whoever designed this trailer needs an Oscar o...,1
3,3,"Let’s admit, this is the best trailer out of a...",1
4,4,The music makes this trailer like 100x more aw...,1


In [3]:
# create pipeline for processing data
# import word tokenizer for tokenization process and pos_tag for speech tagging 
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def preprocess_data(review):
    # Cleaning links
    text = re.sub(r'http\S+', '', review)
    
    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",review)
    
    # Tokenization
    token = word_tokenize(text)
    # tagged = pos_tag(token)
    # keep adjactive (JJ), adverb (RB) and verb (VB)
    # important_tokens = [w for w, pos in tagged if pos.startswith("JJ") or pos.startswith("RB") or pos.startswith("VB") or pos.startswith("NN")]
    preprocessed_text = " ".join(token)
    return preprocessed_text

preprocessed_text = [preprocess_data(text) for text in data["review"]]
print(preprocessed_text)


['8 years later and still one of MARVEL s best teaser trailers', 'Crazy how the line he is a friend from work came from a make a wish kid that was visiting the set while they were shooting the movie The kid said you should say he is a friend from work when you see him and boom Its in the movie Crazy', 'Whoever designed this trailer needs an Oscar or a promotion', 'Let s admit this is the best trailer out of any mcu movie', 'The music makes this trailer like 100x more awesome', 'Led Zeppelin makes everything better True fact', 'This whole marvel universe series of movies is one of the greatest things of my generation', 'I installed a loop plugin for this', 'To this day the best Marvel Teaser Trailer Sets the tone but doesn t give away the surprises', 'This cinematic universe they ve created is marvelous', '1000th like', 'its GLORIOUS', 'Man this still holds up even after watching the movie', 'It certainly coordinated things well to leave impressions The trailer takes careful pains to un

In [4]:
for i in range(0,5):
    print(preprocessed_text[i],end="\n")

8 years later and still one of MARVEL s best teaser trailers
Crazy how the line he is a friend from work came from a make a wish kid that was visiting the set while they were shooting the movie The kid said you should say he is a friend from work when you see him and boom Its in the movie Crazy
Whoever designed this trailer needs an Oscar or a promotion
Let s admit this is the best trailer out of any mcu movie
The music makes this trailer like 100x more awesome


In [5]:
# vectorization
# convert text to vector for SVM
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_text) 
y = np.asarray(data["sentiment"]) # label
# print(vectorizer.get_feature_names_out())
# print(X)
print(y)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [6]:
# split train data and test data and train SVM model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(kernel="linear").fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(X_train.shape)
print(X_train)


              precision    recall  f1-score   support

          -1       0.59      0.83      0.69        24
           0       0.71      0.36      0.48        14
           1       0.82      0.74      0.78        31

    accuracy                           0.70        69
   macro avg       0.71      0.64      0.65        69
weighted avg       0.72      0.70      0.69        69

(273, 1514)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4826 stored elements and shape (273, 1514)>
  Coords	Values
  (0, 29)	0.1567054253800845
  (0, 82)	0.08359540210519067
  (0, 102)	0.31864687710136647
  (0, 110)	0.41196469400970237
  (0, 140)	0.2331632177805134
  (0, 202)	0.11178591937698823
  (0, 249)	0.19147512190732321
  (0, 461)	0.1388706017950529
  (0, 551)	0.19147512190732321
  (0, 559)	0.17608928609026822
  (0, 566)	0.19839352125327467
  (0, 813)	0.1267865118836436
  (0, 860)	0.16213555004805005
  (0, 887)	0.21619786032188582
  (0, 1116)	0.20686095772437824
  (0, 1153)	0.19147512190

In [7]:
# save the model
import joblib

# store the trained model
joblib.dump(model, "../model/svm_sentiment_model.pkl")

# Save the vectorizer (important for preprocessing new text consistently)
# joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['../model/svm_sentiment_model.pkl']

In [11]:
#load the trained model
load_model = joblib.load("../model/svm_sentiment_model.pkl")
# Predict sentiment and test model
new_review = "this seem to be ok, not so intersting"
new_processed = preprocess_data(new_review)
X_new = vectorizer.transform([new_processed])
result = load_model.predict(X_new) # -1 = negative, 1 = positive, 0 = neutral
if result == 1:
    print("Positive")
elif result == 0:
    print("Neural")
else:
    print("Negative")

Neural
